# Super Simple Autometrics Tutorial
=================================

This tutorial shows the absolute basics of using autometrics.
Just load a dataset, run the pipeline, and get your metrics!

## Setup

In [1]:
import os
import dspy
from autometrics.autometrics import Autometrics
from autometrics.dataset.datasets.simplification.simplification import SimpDA
from autometrics.aggregator.regression.ElasticNet import ElasticNet

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = open('/Users/spangher/.openai-reglab-project-key.txt').read().strip()

/Users/spangher/miniconda3/lib/python3.12/site-packages/pyemd/__init__.py:74: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from .emd import emd, emd_with_flow, emd_samples


[Autometrics] No GPU detected - using BM25 + LLMRec pipeline for CPU-optimized performance


In [11]:
dataset

Dataset: SimpDA, Target Columns: ['fluency', 'meaning', 'simplicity'], Ignore Columns: ['id', 'original', 'simple', 'system', 'ref1', 'ref2', 'ref3', 'ref4', 'ref5', 'ref6', 'ref7', 'ref8', 'ref9', 'ref10'], Metric Columns: ['LENS', 'SARI_P', 'SARI_F', 'FKGL', 'BERTScoreP_roberta-large', 'BERTScoreR_roberta-large', 'BERTScoreF_roberta-large', 'BLEU', 'METEOR', 'ROUGE-1-p', 'ROUGE-2-p', 'ROUGE-L-p', 'ROUGE-Lsum-p', 'ROUGE-1-r', 'ROUGE-2-r', 'ROUGE-L-r', 'ROUGE-Lsum-r', 'ROUGE-1-f1', 'ROUGE-2-f1', 'ROUGE-L-f1', 'ROUGE-Lsum-f1', 'distinct_1', 'distinct_2', 'distinct_3', 'distinct_4', 'Perplexity_gpt2-large', 'Autometrics_Regression_simplicity']
   id                                           original  \
0   0  a bastion on the eastern approaches was built ...   
1   0  a bastion on the eastern approaches was built ...   
2   0  a bastion on the eastern approaches was built ...   
3   2  a few animals have chromatic response, changin...   
4   2  a few animals have chromatic response, chan

In [4]:
# Load the SimpDA dataset (text simplification)
dataset = SimpDA(path='../autometrics/dataset/datasets/simplification/simpda.csv')
target_measure = "simplicity"  # The human score column we want to predict

print(f"Dataset: {dataset.get_name()}")
print(f"Size: {len(dataset.get_dataframe())} examples")
print(f"Target measure: {target_measure}")

## Cell 3: Configure LLMs
# Use GPT-4o-mini for both generation and judging
generator_llm = dspy.LM("openai/gpt-4o-mini")
judge_llm = dspy.LM("openai/gpt-4o-mini")

print("LLMs configured!")

Dataset: SimpDA
Size: 434 examples
Target measure: simplicity
LLMs configured!


## Autometrics Pipeline

In [5]:
# Super simple configuration:
# - Generate 1 metric using LLM judge
# - Retrieve 10 metrics from the bank
# - Select top 5 using ElasticNet regression
autometrics = Autometrics(
    metric_generation_configs={
        "llm_judge": {"metrics_per_trial": 1}  # Just generate 1 metric
    },
    regression_strategy=ElasticNet,  # Use ElasticNet instead of default Lasso
    seed=42,  # For reproducibility
    generated_metrics_dir="tutorial_metrics"  # Unique directory for this tutorial
)

In [6]:
print("Running autometrics pipeline...")
print("This will:")
print("1. Generate 1 LLM judge metric")
print("2. Retrieve 10 relevant metrics from the bank")
print("3. Evaluate all metrics on your dataset")
print("4. Select top 5 using ElasticNet regression")
print("5. Create a final aggregated metric")

results = autometrics.run(
    dataset=dataset,
    target_measure=target_measure,
    generator_llm=generator_llm,
    judge_llm=judge_llm,
    num_to_retrieve=10,  # Retrieve 10 metrics
    num_to_regress=5     # Select top 5
)

print("Pipeline complete! 🎉")

Running autometrics pipeline...
This will:
1. Generate 1 LLM judge metric
2. Retrieve 10 relevant metrics from the bank
3. Evaluate all metrics on your dataset
4. Select top 5 using ElasticNet regression
5. Create a final aggregated metric
[Autometrics] Starting pipeline for SimpDA - simplicity
[Autometrics] Configuration: retrieve=10, regress=5, regenerate=False

[Autometrics] Step 1: Generating/Loading Metrics
[Autometrics] Generating 1 metrics using llm_judge...
Initializing BM25 recommender with index path: /Users/spangher/Library/Application Support/autometrics/bm25_all_metrics
Building BM25 index in /Users/spangher/Library/Application Support/autometrics/bm25_all_metrics/index for 48 metrics …


2025-11-17 17:53:54,496 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - Setting log level to INFO
2025-11-17 17:53:54,497 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) - ============ Loading Index Configuration ============
2025-11-17 17:53:54,497 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) - AbstractIndexer settings:
2025-11-17 17:53:54,505 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + DocumentCollection path: /Users/spangher/Library/Application Support/autometrics/bm25_all_metrics/collection
2025-11-17 17:53:54,506 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + CollectionClass: JsonCollection
2025-11-17 17:53:54,506 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:215) -  + Index path: /Users/spangher/Library/Application Support/autometrics/bm25_all_metrics/index
2025-11-17 17:53:54,506 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:216) -  + Threads: 1
2025-11-17 17:53:54,506

Nov 17, 2025 5:53:54 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


2025-11-17 17:53:54,829 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:292) - Indexing Complete! 48 documents indexed
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:293) - ============ Final Counter Values ============
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:294) - indexed:               48
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:295) - unindexable:            0
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:296) - empty:                  0
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:297) - skipped:                0
2025-11-17 17:53:54,830 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:298) - errors:                 0
2025-11-17 17:53:54,833 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:301) - Total 48 documents indexed in 00:00:00
BM25 recommender loaded successfully
[Au

Nov 17, 2025 5:53:54 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
/Users/spangher/Projects/stanford-research/rfi-research/regulations-demo/scripts/autometrics/examples/tutorial_metrics/generated_metrics/SimpDA/simplicity/seed_42/llm_judge/SimpDA_simplicity_llm_judge_seed42_metric01.py:10: SyntaxWarning: invalid escape sequence '\h'
  """---


...

    ✓ Perplexity_gpt2-large computed successfully (parallel)


Predicting DataLoader 0:  10%|████▉                                               | 26/272 [00:53<08:29,  2.07s/it]

    ✓ BERTScore_roberta-large computed successfully (parallel)


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████| 272/272 [07:20<00:00,  1.62s/it]
/Users/spangher/Projects/stanford-research/rfi-research/regulations-demo/scripts/autometrics/autometrics/util/report_card.py:777: SyntaxWarning: invalid escape sequence '\]'
  template = """


    ✓ LENS computed successfully (parallel)
[Autometrics] Aggregating results back to original dataset...
    ✓ LENS aggregated successfully
    ✓ SARI aggregated successfully
    ✓ FKGL aggregated successfully
    ✓ BERTScore_roberta-large aggregated successfully
    ✓ BLEU aggregated successfully
    ✓ METEOR aggregated successfully
    ✓ ROUGE aggregated successfully
    ✓ DistinctNGram aggregated successfully
    ✓ Perplexity_gpt2-large aggregated successfully
[Autometrics] Parallel evaluation complete. Dataset now has 9 metrics
[Autometrics] Successfully evaluated 9 metrics, 0 failed

[Autometrics] Step 5: Regression Analysis (Selecting Top 5 via ElasticNet)
[Autometrics] Running regression to select top 5 metrics from 9 candidates...
  Fitting regression on all metrics to identify importance...
  Metric importance scores:
    1. LENS: 0.3799
    2. BERTScoreP_roberta-large: 0.1846
    3. BLEU: 0.0802
    4. ROUGE-L-p: -0.0473
    5. Perplexity_gpt2-large: 0.0437
    6. ROUGE-1-f1

In [7]:
print("\n" + "="*50)
print("RESULTS")
print("="*50)

print(f"\nGenerated metrics: {len(results['all_generated_metrics'])}")
for i, metric in enumerate(results['all_generated_metrics']):
    print(f"  {i+1}. {metric.__name__}")

print(f"\nRetrieved metrics: {len(results['retrieved_metrics'])}")
for i, metric in enumerate(results['retrieved_metrics'][:3]):  # Show first 3
    print(f"  {i+1}. {metric.__name__}")

print(f"\nTop selected metrics: {len(results['top_metrics'])}")
for i, metric in enumerate(results['top_metrics']):
    print(f"  {i+1}. {metric.get_name()}")

print(f"\nFinal regression metric: {results['regression_metric'].get_name()}")
print(f"Description: {results['regression_metric'].get_description()}")


RESULTS

Generated metrics: 1
  1. Clarity_of_Expression_gpt_4o_mini_LLMJudge

Retrieved metrics: 9
  1. LENS
  2. SARI
  3. FKGL

Top selected metrics: 5
  1. LENS
  2. BERTScore_roberta-large
  3. BLEU
  4. ROUGE
  5. Perplexity_gpt2-large

Final regression metric: Autometrics_Regression_simplicity
Description: Regression aggregator for simplicity using top 5 metrics


## Use Your Metrics

In [8]:
print("\n" + "="*50)
print("USING YOUR METRICS")
print("="*50)

# Get predictions from your final metric
final_scores = results['regression_metric'].predict(dataset)
human_scores = dataset.get_dataframe()[target_measure]

print(f"\nPredicted vs Human scores for first 5 examples:")
print("Example | Predicted | Human | Pred Rank | Human Rank")
print("-" * 55)

# Get first 5 examples
first_5_pred = final_scores[:5]
first_5_human = human_scores.iloc[:5]

for i in range(min(5, len(final_scores))):
    predicted = first_5_pred[i]
    human = first_5_human.iloc[i]
    
    # Calculate ranks within these 5 examples (higher score = higher rank)
    pred_rank = (first_5_pred > predicted).sum() + 1
    human_rank = (first_5_human > human).sum() + 1
    
    print(f"  {i+1}     | {predicted:.3f}    | {human:.3f} | {pred_rank:>9} | {human_rank:>10}")

# Check correlation with human scores
import numpy as np
from scipy.stats import pearsonr

correlation, p_value = pearsonr(human_scores, final_scores)
print(f"\nCorrelation with human scores: {correlation:.3f} (p={p_value:.3f})")


USING YOUR METRICS

Predicted vs Human scores for first 5 examples:
Example | Predicted | Human | Pred Rank | Human Rank
-------------------------------------------------------
  1     | 0.813    | 1.352 |         1 |          1
  2     | -0.111    | -0.873 |         3 |          5
  3     | -0.641    | -0.472 |         4 |          3
  4     | -0.039    | -0.281 |         2 |          2
  5     | -1.013    | -0.684 |         5 |          4

Correlation with human scores: 0.780 (p=0.000)


In [9]:
print("\n" + "="*50)
print("REPORT CARD")
print("="*50)

print(results['report_card'])

print("\n" + "="*50)
print("TUTORIAL COMPLETE!")
print("="*50)
print("You now have:")
print("✅ A custom metric for your task")
print("✅ Top 5 most relevant metrics")
print("✅ A final aggregated metric")
print("✅ Correlation with human judgments")
print("\nYou can use these metrics on new data!")


REPORT CARD

# Autometrics Report Card

## Dataset Information
- **Dataset**: SimpDA
- **Target Measure**: simplicity
- **Dataset Size**: 434 examples

## Top Metrics Selected
- **1.** LENS
- **2.** BERTScore_roberta-large (MultiMetric: BERTScoreP_roberta-large, BERTScoreR_roberta-large, BERTScoreF_roberta-large)
- **3.** BLEU
- **4.** ROUGE (MultiMetric: ROUGE-1-p, ROUGE-2-p, ROUGE-L-p, ROUGE-Lsum-p, ROUGE-1-r, ROUGE-2-r, ROUGE-L-r, ROUGE-Lsum-r, ROUGE-1-f1, ROUGE-2-f1, ROUGE-L-f1, ROUGE-Lsum-f1)
- **5.** Perplexity_gpt2-large

## Regression Aggregator
- **Type**: ElasticNet
- **Name**: Autometrics_Regression_simplicity
- **Description**: Regression aggregator for simplicity using top 5 metrics

## Summary
The Autometrics pipeline successfully identified the most relevant metrics for evaluating simplicity on the SimpDA dataset. The selected metrics can be used individually or combined through the regression aggregator for comprehensive evaluation.

## Hotelling T² Selection
- Selecte